#### Prompt:
You are an analyst for a premium travel agency. The company has tasked you to analyze the recent bookings they had with some of their clients. Each row represents the transaction of a person of interest (POI) in the company. Some details regarding the POI is also given, such as which luxury group they are from, as well as their individual net worth. Most importantly, you also know their __`route`__ which displays the order of their travels from different points A to E. 

A sample route of __`EBDAC`__ means that:
1. From starting point E, they went to point B.
2. From point B, they went to point D.
3. From point D, they went to point A.
4. From point A, they went to ending point C.

#### You are tasked to come up with the following:
1. How many POIs are there in each group?
2. Using the __`route_matrix.json`__ file, calculate each POI's travel expenditure.
3. Due to company policy, add a column named __`budget`__ to denote if a person is overbudget (if their travel expenses go over 5% of their net worth). If they are overbudget, place them as "Over Budget"; else, place them as "Within Budget".
4. Which group has the __highest average net worth__?
5. Which group has the __highest % of people who are Over Budget__?
6. Which group had the most __direct flights__ from point A to point B (or from B to A)?

In [15]:
# 1. How many POIs are there in each group?
import pandas as pd

df = pd.read_csv("poi_dataset.csv")
df

df.groupby('group').count()['poi_number']

group
Group 1     6615
Group 10    6544
Group 11    6603
Group 12    6600
Group 13    6558
Group 14    6564
Group 15    6562
Group 16    6555
Group 17    6445
Group 18    6565
Group 19    6546
Group 2     6533
Group 20    6517
Group 3     6656
Group 4     6660
Group 5     6571
Group 6     6551
Group 7     6501
Group 8     6572
Group 9     6645
Name: poi_number, dtype: int64

In [25]:
df

,poi_number,group,route,net_worth
0,0,Group 1,EBDAC,459
1,1,Group 3,ABCED,407
2,2,Group 9,BECAD,5
3,3,Group 1,BEACD,832
4,4,Group 1,CBDAE,928
...,...,...,...,...
131358,131358,Group 2,AECBD,755
131359,131359,Group 9,EADCB,143
131360,131360,Group 5,CBAED,202
131361,131361,Group 3,ADCEB,277


In [29]:
# 2. Using the route_matrix.json file, calculate each POI's travel expenditure
import json

with open("route_matrix.json","r") as file:
    data = json.load(file)

data

df1 = pd.DataFrame(data)

df1

df['expenses'] = df['route'].apply(lambda x: df1.loc[x[0],x[1]] +  df1.loc[x[1],x[2]] +  df1.loc[x[2],x[3]] +  df1.loc[x[3],x[4]])

df

,poi_number,group,route,net_worth,expenses
0,0,Group 1,EBDAC,459,16
1,1,Group 3,ABCED,407,22
2,2,Group 9,BECAD,5,21
3,3,Group 1,BEACD,832,29
4,4,Group 1,CBDAE,928,8
...,...,...,...,...,...
131358,131358,Group 2,AECBD,755,9
131359,131359,Group 9,EADCB,143,11
131360,131360,Group 5,CBAED,202,18
131361,131361,Group 3,ADCEB,277,17


In [37]:
# 3. Due to company policy, add a column named budget to denote if a person is overbudget (if their travel expenses go over 5% of their net worth). 
# If they are overbudget, place them as "Over Budget"; else, place them as "Within Budget"

df['budget'] = df.apply(lambda x: "Over Budget" if (x['net_worth'] * 0.05) < x['expenses'] else "Within Budget", axis = 1)


df


,poi_number,group,route,net_worth,expenses,budget
0,0,Group 1,EBDAC,459,16,Within Budget
1,1,Group 3,ABCED,407,22,Over Budget
2,2,Group 9,BECAD,5,21,Over Budget
3,3,Group 1,BEACD,832,29,Within Budget
4,4,Group 1,CBDAE,928,8,Within Budget
...,...,...,...,...,...,...
131358,131358,Group 2,AECBD,755,9,Within Budget
131359,131359,Group 9,EADCB,143,11,Over Budget
131360,131360,Group 5,CBAED,202,18,Over Budget
131361,131361,Group 3,ADCEB,277,17,Over Budget


In [39]:
# 4. Which group has the highest average net worth?
df.groupby('group').mean(numeric_only = True)['net_worth'].sort_values(ascending = False)


group
Group 10    510.857427
Group 13    506.817627
Group 9     506.086832
Group 4     506.020871
Group 18    505.240061
Group 19    504.931714
Group 20    504.469695
Group 6     503.275836
Group 3     503.217849
Group 17    502.266253
Group 7     501.841870
Group 2     500.622991
Group 5     500.229037
Group 15    499.502133
Group 1     498.621920
Group 16    497.430206
Group 14    496.734918
Group 12    495.620152
Group 8     494.873098
Group 11    493.068605
Name: net_worth, dtype: float64

In [95]:
# 5. Which group has the highest % of people who are Over Budget?

dummy_df = pd.DataFrame(df.groupby(['group','budget']).count()['route'] / df.groupby(['group']).count()['route'])

dummy_df.reset_index(inplace = True)

dummy_df = dummy_df.sort_values('route', ascending = False)

dummy_df[dummy_df['budget'] == 'Over Budget']




,group,budget,route
0,Group 1,Over Budget,0.367045
6,Group 12,Over Budget,0.366818
4,Group 11,Over Budget,0.364683
36,Group 8,Over Budget,0.364425
12,Group 15,Over Budget,0.363913
30,Group 5,Over Budget,0.358697
34,Group 7,Over Budget,0.358560
10,Group 14,Over Budget,0.358470
14,Group 16,Over Budget,0.358352
22,Group 2,Over Budget,0.357875


In [113]:
# 6. Which group had the most direct flights from A to B (or from B to A)?
madummy_df = df.loc[df['route'].str.contains('AB|BA', regex = True)]

madummy_df['group'].value_counts()

group
Group 3     2742
Group 13    2717
Group 18    2665
Group 12    2659
Group 6     2653
Group 9     2649
Group 19    2640
Group 5     2637
Group 15    2636
Group 2     2629
Group 8     2617
Group 7     2614
Group 1     2611
Group 16    2608
Group 20    2604
Group 10    2602
Group 11    2597
Group 4     2589
Group 14    2574
Group 17    2508
Name: count, dtype: int64